In [ ]:
%pip install --quiet langchain pypdf pymongo langchain-openai tiktoken google-cloud-secret-manager

In [ ]:
import os

%pip show langchain

from platform import python_version
print(python_version())

Name: langchain
Version: 0.1.1
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, async-timeout, dataclasses-json, jsonpatch, langchain-community, langchain-core, langsmith, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 
3.10.12


In [ ]:
from google.cloud import secretmanager
from google.colab import auth
from google.colab import drive

In [ ]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
def load_secrets(secrets_name, project_id):
  # Build a client
  auth.authenticate_user()
  client = secretmanager.SecretManagerServiceClient()
  secret_name = secrets_name
  # Create path to latest secret
  resource_name = f"projects/{project_id}/secrets/{secret_name}/versions/latest"
  # Get your secret :
  response = client.access_secret_version(request={"name": resource_name})
  secret_string = response.payload.data.decode('UTF-8')
  return secret_string

In [ ]:
project_id = 'botchagalupep1'
openai_api_key = load_secrets("openai_api_key",project_id)
os.environ['OPENAI_API_KEY'] = openai_api_key
#MONGODB_ATLAS_CLUSTER_URI = load_secrets("mdb_uri",project_id)
MONGODB_ATLAS_CLUSTER_URI = load_secrets("mdb_uri",project_id)
langsmith_api_key = load_secrets("langsmith_api_key",project_id)

Now we will setup the environment variables for the MongoDB Atlas cluster


In [ ]:
from pymongo import MongoClient

# initialize MongoDB python client
client = MongoClient(MONGODB_ATLAS_CLUSTER_URI)

DB_NAME = "langchain_db"
COLLECTION_NAME = "test"
ATLAS_VECTOR_SEARCH_INDEX_NAME = "opencontext-2"

MONGODB_COLLECTION = client[DB_NAME][COLLECTION_NAME]

 In the below example, embedding is the name of the field that contains the embedding vector. Please refer to the documentation to get more details on how to define an Atlas Vector Search index. You can name the index {ATLAS_VECTOR_SEARCH_INDEX_NAME} and create the index on the namespace {DB_NAME}.{COLLECTION_NAME}. Finally, write the following definition in the JSON editor on MongoDB Atlas:

~~~
{
 "fields": [{
   "type": "vector",
   "path": "embedding",
   "numDimensions": 1536,
   "similarity": "cosine"
 }]
}



#Load Data

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

# Load the PDF
#loader = PyPDFLoader("https://arxiv.org/pdf/2303.08774.pdf")
#loader = PyPDFLoader("/content/gdrive/MyDrive/GAI/transcripts/DEM.pdf")
#loader = PyPDFLoader("/content/gdrive/MyDrive/GAI/transcripts/dev-final-entities-db.pdf")
loader = PyPDFLoader("/content/gdrive/MyDrive/GAI/transcripts/scatterly-parsed-json.pdf")
data = loader.load()

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
docs = text_splitter.split_documents(data)

In [ ]:
print(docs[0])

page_content='scatterly-parsed-jsonkindentity_typeentity_othertypeentity_uidentity_nameentity_namespaceentity_descriptionentity_etag AuxComponentdocumentNULLf7ce8b98-59f1-4db2-9b2b-d9d451b3fce2barrow-frontend-maintenancedefaultMaintenance documentationeb1f2e13e4f23e78db46d009d6557db52e9ad314 AuxComponentdocumentNULLa3b077c0-34ae-49cd-b4c6-3465d593e56dbarrow-frontend-maintenancescatterly-glMaintenance documentation9ce76965b20454c5506ace2576bbc6d923432d62 AuxComponentdocumentNULLfbcd9000-f10d-4881-8642-1c5abaa93bdacrates-erp-ownershipscatter-lyCrates ERP Service Ownership documentation11f0281ba7ce26a1a1bfc78ae87edfe3abc983fd AuxComponentdocumentNULL885bce3f-5caf-4eb0-8c04-280db5f28b66crates-erp-ownershipscatterly-glCrates ERP Service Ownership documentation5e87885d8ef35121c0f0cd397ef423fd54f44b5e AuxComponentdocumentNULL69c740dc-a9fd-4599-bd24-aaaf852248c6crates-erp-rfcscatter-lyCrates ERP Architectural RFC3c92434077885936694299ab7358d41b6a5b057e' metadata={'source': '/content/gdrive/MyD

In [ ]:
from langchain_community.vectorstores import MongoDBAtlasVectorSearch
from langchain_openai import OpenAIEmbeddings

# insert the documents in MongoDB Atlas with their embedding
vector_search = MongoDBAtlasVectorSearch.from_documents(
    documents=docs,
    embedding=OpenAIEmbeddings(disallowed_special=()),
    collection=MONGODB_COLLECTION,
    index_name=ATLAS_VECTOR_SEARCH_INDEX_NAME,
)

In [ ]:
# Perform a similarity search between the embedding of the query and the embeddings of the documents
#query = "What were the compute requirements for training GPT 4"
query = "what are the code componets in this repository"
#query = "what code compenents are typescript?"
results = vector_search.similarity_search(query)

print(results[0].page_content)

icon:"github"}]tags:["typescript"]uid:"64c2b4af-37fc-4c75-924b-e66d2e2ad34b"etag:"21ae3f170f75c0c6660486bcfab7e85c3bc71f0e"}apiVersion:"opencontext.com/v1alpha1"kind:"CodeComponent"spec:{"type":"repository"
